In [1]:
from pathlib import Path

from src.metric import cls_eval_funs, reg_eval_funs
from src.trainer import Trainer
from src.utils import load_pkl_data, set_random_seed

In [13]:
class Config:
    data_dir = '/home/jiawei/Desktop/github/DOFEN/tabular-benchmark/tabular_benchmark_data'
    data_id = '361060'
    n_epoch = 250
    batch_size = 256
    target_transform = False

In [85]:
from typing import Dict, List, Optional

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

from src.models.base import BaseClassifier, BaseRegressor
from src.models.layers import ResidualLayer, ReGLU


class Reshape(nn.Module):
    def __init__(self, *args: int) -> None:
        super(Reshape, self).__init__()
        self.shape = args

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x.reshape(self.shape)


class FastGroupConv1d(nn.Conv1d):
    def __init__(self, *args, **kwargs):
        self.fast_mode = kwargs.pop('fast_mode')
        nn.Conv1d.__init__(self, *args, **kwargs)
        if self.groups > self.fast_mode:
            self.weight = nn.Parameter(
                self.weight.reshape(
                    self.groups, self.out_channels // self.groups, self.in_channels // self.groups, 1
                ).permute(3, 0, 2, 1)
            )
            self.bias = nn.Parameter(
                self.bias.unsqueeze(0).unsqueeze(-1)
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.groups > self.fast_mode:
            x = x.reshape(-1, self.groups, self.in_channels // self.groups, 1)
            return (x * self.weight).sum(2, keepdims=True).permute(0, 1, 3, 2).reshape(-1, self.out_channels, 1) + self.bias
        else:
            return self._conv_forward(x, self.weight, self.bias)


class ConditionGeneration(nn.Module):
    def __init__(
        self,
        category_column_count: List[int],
        n_cond: int = 128,
        categorical_optimized: bool = False,
        fast_mode: int = 64,
    ):
        super(ConditionGeneration, self).__init__()
        self.fast_mode = fast_mode
        self.categorical_optimized = categorical_optimized

        index_info = self.extract_feature_metadata(category_column_count)
        self.numerical_index = index_info['numerical_index']
        self.categorical_index = index_info['categorical_index']
        self.categorical_count = index_info['categorical_count']

        categorical_offset = torch.tensor([0] + np.cumsum(self.categorical_count).tolist()[:-1]).long()
        self.register_buffer('categorical_offset', categorical_offset)

        self.n_cond = n_cond
        self.phi_1 = self.get_phi_1()

    def extract_feature_metadata(self, category_column_count: List[int]) -> Dict[str, List[int]]:
        numerical_index = [i for i, count in enumerate(category_column_count) if count == -1]
        categorical_index = [i for i, count in enumerate(category_column_count) if count != -1]
        categorical_count = [count for count in category_column_count if count != -1]
        return {
            'numerical_index': numerical_index,
            'categorical_index': categorical_index,
            'categorical_count': categorical_count,
        }

    def get_phi_1(self) -> nn.ModuleDict:
        phi_1 = nn.ModuleDict()
        if len(self.numerical_index):
            phi_1['num'] = nn.Sequential(
                # input = (b, n_num_col)
                # output = (b, n_num_col, n_cond)
                Reshape(-1, len(self.numerical_index), 1),
                FastGroupConv1d(
                    len(self.numerical_index),
                    len(self.numerical_index) * self.n_cond,
                    kernel_size=1,
                    groups=len(self.numerical_index),
                    fast_mode=self.fast_mode
                ), # (b, n_num_col, 1) -> (b, n_num_col * n_cond, 1)
                nn.Sigmoid(),
                Reshape(-1, len(self.numerical_index), self.n_cond)
            )
        if len(self.categorical_index):
            phi_1['cat'] = nn.ModuleDict()
            phi_1['cat']['embedder'] = nn.Embedding(sum(self.categorical_count), self.n_cond)            
            phi_1['cat']['mapper'] = nn.Sequential(
                # input = (b, n_cat_col, n_cond)
                # output = (b, n_cat_col, n_cond)
                Reshape(-1, len(self.categorical_index) * self.n_cond, 1),
                nn.GroupNorm(len(self.categorical_index), len(self.categorical_index) * self.n_cond),
                FastGroupConv1d(len(self.categorical_index) * self.n_cond, len(self.categorical_index)*self.n_cond, kernel_size=1, groups=len(self.categorical_index)*self.n_cond if self.categorical_optimized else len(self.categorical_index), fast_mode=self.fast_mode),                
                nn.Sigmoid(),
                Reshape(-1, len(self.categorical_index), self.n_cond)
            )
        return phi_1

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        M = []

        if len(self.numerical_index):
            num_x = x[:, self.numerical_index].float()
            num_sample_emb = self.phi_1['num'](num_x)
            M.append(num_sample_emb)

        if len(self.categorical_index):
            cat_x = x[:, self.categorical_index].long() + self.cat_offset
            cat_sample_emb = self.phi_1['cat']['mapper'](self.phi_1['cat']['embedder'](cat_x))
            M.append(cat_sample_emb)

        M = torch.cat(M, dim=1) # (b, n_col, n_cond)
        M = M.permute(0, 2, 1) # (b, n_cond, n_col)
        return M


class rODTConstruction(nn.Module):
    def __init__(self, n_cond: int, n_col: int, d: int) -> None:
        super().__init__()
        self.permutator = torch.rand(n_cond * n_col).argsort(-1)
        self.d = d

    def forward(self, M: torch.Tensor) -> torch.Tensor:
        b, _, _, embed_dim = M.shape
        return M.reshape(b, -1, embed_dim)[:, self.permutator, :].reshape(b, -1, self.d, embed_dim)


class rODTForestConstruction(nn.Module):
    def __init__(
        self,
        n_col: int,
        n_rodt: int,
        n_cond: int,
        n_estimator: int,
        n_head: int = 1,
        n_hidden: int = 128,
        n_forest: int = 100,
        dropout: float = 0.0,
        fast_mode: int = 64,
        device = torch.device('cuda')
    ) -> None:

        super().__init__()
        self.device = device
        self.n_estimator = n_estimator
        self.n_forest = n_forest
        self.n_rodt = n_rodt
        self.n_head = n_head
        self.n_hidden = n_hidden

        self.sample_without_replacement_eval = self.get_sample_without_replacement()

    def get_sample_without_replacement(self) -> torch.Tensor:
        return torch.rand(self.n_forest, self.n_rodt, device=self.device).argsort(-1)[:, :self.n_estimator]

    def forward(self, w, E) -> torch.Tensor:
        # w: (b, n_rodt, 1)
        # E: (b, n_rodt, n_hidden)
    
        sample_without_replacement = self.get_sample_without_replacement() if self.training else self.sample_without_replacement_eval

        w_prime = w[:, sample_without_replacement].softmax(-2) # (b, n_forest, n_rodt, 1)
        E_prime = E[:, sample_without_replacement].reshape(
            E.shape[0], self.n_forest, self.n_estimator, self.n_hidden
        ) # (b, n_forest, n_rodt, n_hidden)

        F = (w_prime * E_prime).sum(-2).reshape(
            E.shape[0], self.n_forest, self.n_hidden
        ) # (b, n_forest, n_hidden)
        return F


class rODTForestBagging(nn.Module):
    def __init__(self, n_hidden: int, dropout: float, n_class: int) -> None:
        super().__init__()
        self.phi_3 = nn.Sequential(
            nn.LayerNorm(n_hidden),
            nn.Dropout(dropout),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.LayerNorm(n_hidden),
            nn.Dropout(dropout),
            nn.Linear(n_hidden, n_class)
        )

    def forward(self, F: torch.Tensor) -> torch.Tensor:
        return self.phi_3(F) # (b, n_forest, n_class)


class MultiheadAttention(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int, dropout: float = 0.):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scaling = self.head_dim ** -0.5
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"

        self.qw_proj = self.create_linear(embed_dim, embed_dim)
        self.kw_proj = self.create_linear(embed_dim, embed_dim)
        self.vw_proj = self.create_linear(embed_dim, embed_dim)
        self.ow_proj = self.create_linear(embed_dim, 1)

        self.qE_proj = self.create_linear(embed_dim, embed_dim)
        self.kE_proj = self.create_linear(embed_dim, embed_dim)
        self.vE_proj = self.create_linear(embed_dim, embed_dim)
        self.oE_proj = self.create_linear(embed_dim, embed_dim)

        self.dropout = nn.Dropout(dropout)

    def create_linear(self, in_features: int, out_features: int):
        linear = nn.Linear(in_features, out_features)
        nn.init.xavier_uniform_(linear.weight, gain=1 / 2 ** 0.5)
        return linear

    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor) -> torch.Tensor:
        """
        query: Tensor of shape (batch_size, tgt_len, embed_dim)
        key: Tensor of shape (batch_size, src_len, embed_dim)
        value: Tensor of shape (batch_size, src_len, embed_dim)
        attn_mask: Optional[Tensor] of shape (tgt_len, src_len) or (batch_size, tgt_len, src_len)
        """
        batch_size, tgt_len, _ = query.size()
        src_len = key.size(1)

        # Compute w
        qw = self.qw_proj(query)  # shape: (batch_size, tgt_len, embed_dim)
        kw = self.kw_proj(key)    # shape: (batch_size, src_len, embed_dim)
        vw = self.vw_proj(value)  # shape: (batch_size, src_len, 1)

        # Reshape into multihead format
        qw = qw.view(batch_size, tgt_len, self.num_heads, self.head_dim).transpose(1, 2) # shape: (batch_size, num_heads, tgt_len, head_dim)
        kw = kw.view(batch_size, src_len, self.num_heads, self.head_dim).transpose(1, 2) # shape: (batch_size, num_heads, src_len, head_dim)
        vw = vw.view(batch_size, src_len, self.num_heads, self.head_dim).transpose(1, 2) # shape: (batch_size, num_heads, src_len, 1)

        attn_w = F.softmax(torch.matmul(qw, kw.transpose(-2, -1)) / self.scaling, dim=-1) # shape: (batch_size, num_heads, tgt_len, src_len)
        attn_w = self.dropout(attn_w) 
        attn_w_output = torch.matmul(attn_w, vw) # shape: (batch_size, num_heads, tgt_len, head_dim)

        # Reshape back to original dimensions
        attn_w_output = attn_w_output.transpose(1, 2).contiguous().view(batch_size, tgt_len, self.embed_dim) # shape: (batch_size, tgt_len, embed_dim)
        w_output = self.ow_proj(attn_w_output) # shape: (batch_size, tgt_len, 1)
        w_output = w_output.mean(1)

        # Compute w
        qE = self.qE_proj(query)  # shape: (batch_size, tgt_len, embed_dim)
        kE = self.kE_proj(key)    # shape: (batch_size, src_len, embed_dim)
        vE = self.vE_proj(value)  # shape: (batch_size, src_len, embed_dim)

        # Reshape into multihead format
        qE = qE.view(batch_size, tgt_len, self.num_heads, self.head_dim).transpose(1, 2) # shape: (batch_size, num_heads, tgt_len, head_dim)
        kE = kE.view(batch_size, src_len, self.num_heads, self.head_dim).transpose(1, 2) # shape: (batch_size, num_heads, src_len, head_dim)
        vE = vE.view(batch_size, src_len, self.num_heads, self.head_dim).transpose(1, 2) # shape: (batch_size, num_heads, src_len, 1)

        attn_E = F.softmax(torch.matmul(qE, kE.transpose(-2, -1)) / self.scaling, dim=-1) # shape: (batch_size, num_heads, tgt_len, src_len)
        attn_E = self.dropout(attn_E) 
        attn_E_output = torch.matmul(attn_E, vE) # shape: (batch_size, num_heads, tgt_len, head_dim)

        # Reshape back to original dimensions
        attn_E_output = attn_E_output.transpose(1, 2).contiguous().view(batch_size, tgt_len, self.embed_dim) # shape: (batch_size, tgt_len, embed_dim)
        E_output = self.oE_proj(attn_E_output) # shape: (batch_size, tgt_len, embed_dim)
        E_output = E_output.mean(1)

        return w_output, E_output


class DOFEN(nn.Module):
    def __init__(
        self,
        category_column_count: List[int],
        n_class: int, 
        m: int = 16, 
        d: int = 4, 
        n_head: int = 1,
        n_forest: int = 100,
        n_hidden: int = 128,
        dropout: float = 0.0, 
        categorical_optimized: bool = False,
        fast_mode: int = 2048,
        use_bagging_loss: bool = False,
        device=torch.device('cuda'),
    ):
        super().__init__()

        self.device = device
        self.n_class = 1 if n_class == -1 else n_class
        self.is_rgr = True if n_class == -1 else False

        self.m = m
        self.d = d
        self.n_head = n_head
        self.n_forest = n_forest
        self.n_hidden = n_hidden
        self.dropout = dropout
        self.use_bagging_loss = use_bagging_loss

        self.n_cond = self.d * self.m
        self.n_col = len(category_column_count)
        self.n_rodt = self.n_cond * self.n_col // self.d
        self.n_estimator = max(2, int(self.n_col ** 0.5)) * self.n_cond // self.d

        self.condition_generation = ConditionGeneration(            
            category_column_count, 
            n_cond=self.n_cond, 
            categorical_optimized=categorical_optimized, 
            fast_mode=fast_mode,
        )

        self.proj = nn.Linear(1, self.n_hidden)
        self.rodt_construction = rODTConstruction(
            self.n_cond,
            self.n_col,
            self.d,
        )

        self.norm = nn.LayerNorm(n_hidden)
        self.attn = MultiheadAttention(
            embed_dim=self.n_hidden,
            num_heads=n_head,
            dropout=0.2,
        )

        self.rodt_forest_construction = rODTForestConstruction(
            self.n_col, 
            self.n_rodt, 
            self.n_cond,
            self.n_estimator,
            n_head=self.n_head, 
            n_hidden=self.n_hidden,
            n_forest=self.n_forest,
            dropout=self.dropout,
            fast_mode=fast_mode,
            device=self.device
        )
        self.rodt_forest_bagging = rODTForestBagging(
            self.n_hidden,
            self.dropout,
            self.n_class
        )

        self.ffn_dropout_rate = 0.1
        self.act = ReGLU()

    def compute_loss(self, y_hat: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        raise NotImplementedError

    def evaluate(self, X: torch.Tensor, y: torch.Tensor) -> Dict[str, torch.Tensor]:
        raise NotImplementedError

    def forward(self, x: torch.Tensor, y: Optional[torch.Tensor] = None):
        M = self.condition_generation(x)           # (b, n_cond, n_col)
        M = M.unsqueeze(-1)                        # (b, n_cond, n_col, 1)
        M = self.proj(M)                           # (b, n_cond, n_col, n_hidden)
        O = self.rodt_construction(M)              # (b, n_rodt, d, n_hidden)
        O = O.reshape(-1, self.d, self.n_hidden)   # (b * n_rodt, d, n_hidden)
        O = self.norm(O)
        w, E = self.attn(O, O, O)

        w = w.reshape(-1, self.n_rodt, 1)
        E = E.reshape(-1, self.n_rodt, self.n_hidden)

        E_norm = E / E.norm(dim=-1, keepdim=True)
        #sim_loss = torch.einsum('bij,bjk->bik', E_norm, E_norm.permute(0, 2, 1))

        F = self.rodt_forest_construction(w, E) # (b, n_forest, n_hidden)
        y_hats = self.rodt_forest_bagging(F)    # (b, n_rodt, n_class)
        y_hat = y_hats.mean(1)                  # (b, n_class)

        if y is not None:
            loss = self.compute_loss(
                y_hats.permute(0, 2, 1) if not self.is_rgr else y_hats, 
                y.unsqueeze(-1).expand(-1, self.n_forest)
            ) #+  sim_loss.abs().mean()
            if self.n_forest > 1 and self.training and self.use_bagging_loss:
                loss += self.compute_loss(y_hat, y)
            return {'pred': y_hat, 'loss': loss}
        return {'pred': y_hat}

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        return self.forward(x)['pred']


class DOFENClassifier(BaseClassifier, DOFEN):
    def __init__(
        self,
        category_column_count: List[int],
        n_class: int, 
        m: int = 16,
        d: int = 4,
        n_head: int = 4,
        n_forest: int = 100,
        n_hidden: int = 16,
        dropout: float = 0.0, 
        categorical_optimized: bool = False,
        fast_mode: int = 2048,
        use_bagging_loss: bool = False,
        device=torch.device('cuda'),
    ) -> None:

        super().__init__(
            category_column_count=category_column_count, 
            n_class=n_class, 
            m=m, 
            d=d, 
            n_head=n_head,
            n_forest=n_forest,
            n_hidden=n_hidden,
            dropout=dropout, 
            categorical_optimized=categorical_optimized,
            fast_mode=fast_mode,
            use_bagging_loss=use_bagging_loss,
            device=device,
        )

In [86]:
args = Config()
set_random_seed()

data_dict = load_pkl_data(Path(args.data_dir, args.data_id, '0.pkl'))
n_class = data_dict['label_cat_count']
task = 'r' if n_class == -1 else 'c'

target_transform = args.target_transform if task == 'r' else False
train_X = data_dict['x_train']
train_y = data_dict['y_train' if not target_transform else 'y_train_transform']
valid_X = data_dict['x_val']
valid_y = data_dict['y_val' if not target_transform else 'y_val_transform']
test_X = data_dict['x_test']
test_y = data_dict['y_test' if not target_transform else 'y_test_transform']

data_args = {
    'n_feature': train_X.shape[1],
    'n_train': train_X.shape[0],
    'n_valid': valid_X.shape[0],
    'n_test': test_X.shape[0],
}

model_params = {
    'category_column_count': data_dict['col_cat_count'],
}
if n_class != -1:
    model_params['n_class'] = n_class


model = DOFENClassifier(**model_params)
eval_funs = cls_eval_funs
metrics = 'accuracy'

wandb = None
trainer = Trainer(
    model,
    batch_size=args.batch_size,
    n_epoch=args.n_epoch,
    eval_funs=eval_funs,
    metric=metrics,
    logger=wandb,
)
trainer.fit(
    train_X=train_X, train_y=train_y,
    valid_X=valid_X, valid_y=valid_y,
    test_X=test_X, test_y=test_y,
)

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 1/250 [00:01<05:34,  1.34s/it]

{'epoch': 1, 'valid_accuracy': 0.5237649262467806, 'best_valid_epoch': 1, 'best_valid_score': 0.5237649262467806, 'test_accuracy': 0.5192153321292394, 'selected_test_score': 0.5192153321292394}


  1%|          | 2/250 [00:02<05:34,  1.35s/it]

{'epoch': 2, 'valid_accuracy': 0.520135799578553, 'best_valid_epoch': 1, 'best_valid_score': 0.5237649262467806, 'test_accuracy': 0.5195665261890428, 'selected_test_score': 0.5192153321292394}


...

{'epoch': 248, 'valid_accuracy': 0.7793256848513229, 'best_valid_epoch': 245, 'best_valid_score': 0.7849449777569656, 'test_accuracy': 0.779951836243227, 'selected_test_score': 0.7835641180012041}


100%|█████████▉| 249/250 [05:28<00:01,  1.39s/it]

{'epoch': 249, 'valid_accuracy': 0.7800280964645282, 'best_valid_epoch': 245, 'best_valid_score': 0.7849449777569656, 'test_accuracy': 0.7798013245033113, 'selected_test_score': 0.7835641180012041}


100%|██████████| 250/250 [05:30<00:00,  1.32s/it]

{'epoch': 250, 'valid_accuracy': 0.7811987824865371, 'best_valid_epoch': 245, 'best_valid_score': 0.7849449777569656, 'test_accuracy': 0.7798514950832831, 'selected_test_score': 0.7835641180012041}


In [87]:
model

DOFENClassifier(
  (condition_generation): ConditionGeneration(
    (phi_1): ModuleDict(
      (num): Sequential(
        (0): Reshape()
        (1): FastGroupConv1d(7, 448, kernel_size=(1,), stride=(1,), groups=7)
        (2): Sigmoid()
        (3): Reshape()
      )
    )
  )
  (proj): Linear(in_features=1, out_features=16, bias=True)
  (rodt_construction): rODTConstruction()
  (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (attn): MultiheadAttention(
    (qw_proj): Linear(in_features=16, out_features=16, bias=True)
    (kw_proj): Linear(in_features=16, out_features=16, bias=True)
    (vw_proj): Linear(in_features=16, out_features=16, bias=True)
    (ow_proj): Linear(in_features=16, out_features=1, bias=True)
    (qE_proj): Linear(in_features=16, out_features=16, bias=True)
    (kE_proj): Linear(in_features=16, out_features=16, bias=True)
    (vE_proj): Linear(in_features=16, out_features=16, bias=True)
    (oE_proj): Linear(in_features=16, out_features=16, bias=True)

In [88]:
model.attn

MultiheadAttention(
  (qw_proj): Linear(in_features=16, out_features=16, bias=True)
  (kw_proj): Linear(in_features=16, out_features=16, bias=True)
  (vw_proj): Linear(in_features=16, out_features=16, bias=True)
  (ow_proj): Linear(in_features=16, out_features=1, bias=True)
  (qE_proj): Linear(in_features=16, out_features=16, bias=True)
  (kE_proj): Linear(in_features=16, out_features=16, bias=True)
  (vE_proj): Linear(in_features=16, out_features=16, bias=True)
  (oE_proj): Linear(in_features=16, out_features=16, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [89]:
model.cuda()(torch.tensor(train_X[:256]).cuda())

{'pred': tensor([[ 0.2512, -0.0253],
         [-0.4203,  0.9439],
         [ 0.0696,  0.2283],
         [ 0.4746, -0.3041],
         [ 0.8893, -0.8632],
         [ 0.7030, -0.6088],
         [ 0.8161, -0.7506],
         [ 1.2206, -1.3292],
         [ 0.3068, -0.0989],
         [ 1.1070, -1.1566],
         [ 0.4825, -0.3156],
         [-0.3567,  0.8564],
         [ 1.0180, -1.0287],
         [-0.9935,  1.7610],
         [ 0.0696,  0.2234],
         [ 0.2868, -0.0663],
         [ 0.5032, -0.3719],
         [ 0.3093, -0.1194],
         [ 0.2660, -0.0410],
         [-0.8812,  1.6197],
         [-0.9912,  1.7624],
         [ 1.0766, -1.1170],
         [ 0.6953, -0.6056],
         [-0.7315,  1.4160],
         [ 0.2244,  0.0132],
         [ 1.1348, -1.2020],
         [ 0.0075,  0.3226],
         [-0.6082,  1.2394],
         [-0.1344,  0.5107],
         [ 0.6661, -0.5536],
         [ 0.6013, -0.4703],
         [-0.0216,  0.3661],
         [ 1.0307, -1.0517],
         [ 1.4218, -1.6203],
      

In [90]:
activations = {}

def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

handle = model.get_submodule("attn.oE_proj").register_forward_hook(get_activation("oE_proj"))

In [91]:
model.cuda()(torch.tensor(train_X[:256]).cuda())

{'pred': tensor([[ 0.2512, -0.0253],
         [-0.4203,  0.9439],
         [ 0.0696,  0.2283],
         [ 0.4746, -0.3041],
         [ 0.8893, -0.8632],
         [ 0.7030, -0.6088],
         [ 0.8161, -0.7506],
         [ 1.2206, -1.3292],
         [ 0.3068, -0.0989],
         [ 1.1070, -1.1566],
         [ 0.4825, -0.3156],
         [-0.3567,  0.8564],
         [ 1.0180, -1.0287],
         [-0.9935,  1.7610],
         [ 0.0696,  0.2234],
         [ 0.2868, -0.0663],
         [ 0.5032, -0.3719],
         [ 0.3093, -0.1194],
         [ 0.2660, -0.0410],
         [-0.8812,  1.6197],
         [-0.9912,  1.7624],
         [ 1.0766, -1.1170],
         [ 0.6953, -0.6056],
         [-0.7315,  1.4160],
         [ 0.2244,  0.0132],
         [ 1.1348, -1.2020],
         [ 0.0075,  0.3226],
         [-0.6082,  1.2394],
         [-0.1344,  0.5107],
         [ 0.6661, -0.5536],
         [ 0.6013, -0.4703],
         [-0.0216,  0.3661],
         [ 1.0307, -1.0517],
         [ 1.4218, -1.6203],
      

In [92]:
oE_proj_output = activations["oE_proj"]
print(oE_proj_output.shape)

torch.Size([28672, 4, 16])


In [93]:
oE_proj_output = oE_proj_output.reshape(256, -1, 4, 16)

In [94]:
oE_proj_output.shape

torch.Size([256, 112, 4, 16])

In [95]:
x = oE_proj_output[0].view(112, -1)

In [96]:
x_ = x / ((x ** 2).sum(dim=1) ** 0.5)[:, None]
sim = x_ @ x_.T
sim

tensor([[1.0000, 0.9864, 0.9966,  ..., 0.8331, 0.9967, 0.9446],
        [0.9864, 1.0000, 0.9699,  ..., 0.7317, 0.9705, 0.8786],
        [0.9966, 0.9699, 1.0000,  ..., 0.8756, 1.0000, 0.9684],
        ...,
        [0.8331, 0.7317, 0.8756,  ..., 1.0000, 0.8742, 0.9683],
        [0.9967, 0.9705, 1.0000,  ..., 0.8742, 1.0000, 0.9677],
        [0.9446, 0.8786, 0.9684,  ..., 0.9683, 0.9677, 1.0000]],
       device='cuda:0')

In [97]:
sim[0]

tensor([1.0000, 0.9864, 0.9966, 0.9950, 0.9987, 0.9916, 0.8117, 0.9820, 0.8812,
        0.9980, 0.8736, 0.9975, 0.9953, 0.7756, 0.8760, 0.9834, 0.8134, 0.9957,
        0.9969, 0.9995, 0.9883, 0.9851, 0.9917, 0.9916, 0.9787, 0.8838, 0.9921,
        0.9646, 0.9666, 0.9698, 0.9916, 0.9912, 0.9725, 0.8803, 0.8372, 0.9318,
        0.9742, 0.8234, 0.9980, 0.9429, 0.9838, 0.9895, 0.8261, 0.9684, 0.9800,
        0.7999, 0.8236, 0.7952, 0.7878, 0.9967, 0.9602, 0.9996, 0.9560, 0.8429,
        0.9823, 0.9934, 0.9196, 0.9720, 0.8384, 0.9974, 0.9914, 0.9196, 0.9915,
        0.9987, 0.9260, 0.9399, 0.9613, 0.9955, 0.9972, 0.9868, 0.8621, 0.9961,
        0.9988, 0.9552, 0.9926, 0.9950, 0.9754, 0.8788, 0.9974, 0.8764, 0.9973,
        0.8756, 0.7560, 0.9939, 0.9600, 0.9306, 0.9128, 0.9945, 0.9986, 0.9579,
        0.8050, 0.8734, 0.8130, 0.9218, 0.9924, 0.9981, 0.9805, 0.8486, 0.9330,
        0.9673, 0.8925, 0.9559, 0.9933, 0.9302, 0.8561, 0.8310, 0.8679, 0.9961,
        0.9639, 0.8331, 0.9967, 0.9446],